read pkl file and make dict

In [16]:
import pickle

with open("./data/new_programmers.pkl", "rb") as f:
    main = pickle.load(f)

main.columns
newmain = main.to_dict()

Index(['id', 'address', 'career', 'careerRange', 'companyId', 'jobType',
       'maxSalary', 'minSalary', 'personalized', 'signingBonus', 'status',
       'title', 'startAt', 'createdAt', 'updatedAt', 'careerOption',
       'job_position', 'period', 'technicalTags', 'teamTechnicalTags', 'url',
       'isSynced', 'company', 'company_name', 'additionalInformation',
       'description', 'preferredExperience', 'requirement', 'reqired_skills'],
      dtype='object')

company 컬럼에서 회사 정보 뽑아오기 + df에 추가

In [53]:
com_info = list(newmain['company'].values())
com_info
# company 컬럼 안의 values 중 servicename을 key로 가진 값을 new tag에 append
new_tag= []
for i in com_info:
    new_tag.append(i['serviceName'])

#company_description이라는 new column 생성 후 new tag 값 추가
main['company_description'] = new_tag
#df 정리(컬럼 순서 변경, 필요 없는 컬럼 drop)
pro_main = main[['career', 'title', 'job_position', 'period','company_name', 'company_description', 'additionalInformation',\
                 'description','preferredExperience', 'requirement','reqired_skills']]
pro_main.drop(['job_position'], axis=1, inplace=True)
pro_main.columns
pro_new = pro_main[['company_name','title', 'career', 'period', 'reqired_skills', 'description', \
                    'company_description', 'requirement', 'preferredExperience', 'additionalInformation']]
pro_new = pro_new.rename(columns={'reqired_skills' : 'tech_stack', 'period' : 'date', 'requirement' : 'required',\
                                   'preferredExperience' : 'preferred_skill', 'additionalInformation' : 'welfare'})


pro_new.loc[pro_new['tech_stack'].apply(len) == 0, 'tech_stack'] = None #tech_stack열의 빈리스트 값들을 None으로 처리

[{'id': 7697,
  'name': '웅진',
  'address': '서울 중구 청계천로 24, (다동) 케이스퀘어시티 7층 ',
  'serviceName': 'SAP ERP ,AWS/MS클라우드, 세일즈포스, 그룹웨어 등 IT컨설팅 및  대외고객사 시스템 SI/SM , 웅진그룹 IT시스템운영 등',
  'serviceUrl': 'www.woongjin.co.kr',
  'homeUrl': 'https://www.woongjin.co.kr/',
  'funding': 141.0,
  'hideFunding': True,
  'revenue': 141.0,
  'hideRevenue': True,
  'blog': '',
  'countryCode': 'KR',
  'averageResponseTime': '1주',
  'employeesCount': 808,
  'logoUrl': 'https://grepp-programmers.s3.amazonaws.com/production/company/logo/7697/CI_%EC%BB%AC%EB%9F%AC_%EA%B0%80%EB%A1%9C%ED%98%95.png',
  'developers': []},
 {'id': 7697,
  'name': '웅진',
  'address': '서울 중구 청계천로 24, (다동) 케이스퀘어시티 7층 ',
  'serviceName': 'SAP ERP ,AWS/MS클라우드, 세일즈포스, 그룹웨어 등 IT컨설팅 및  대외고객사 시스템 SI/SM , 웅진그룹 IT시스템운영 등',
  'serviceUrl': 'www.woongjin.co.kr',
  'homeUrl': 'https://www.woongjin.co.kr/',
  'funding': 141.0,
  'hideFunding': True,
  'revenue': 141.0,
  'hideRevenue': True,
  'blog': '',
  'countryCode': 'KR',
  'averageResponseTim

Date 컬럼 데이터 변경

In [ ]:
import re
from datetime import datetime
import pandas as pd

# 상시 채용 -> 수시 채용으로 변경
pro_new.loc[pro_new['date'].str.contains('상시 채용'), 'date'] = '수시 채용'

# 'date' 열에서 날짜 추출해서 변경 후 원본 데이터프레임에 저장
# "2024-04-18 01:49 부터 2024-05-31 14:59까지" 형태의 데이터를 %Y-%m-%d 형태로 변환하는 과정

pattern = r'\d{4}-\d{2}-\d{2}'  # 'yyyy-mm-dd' 형식의 패턴

def extract_date(row):
    dates = re.findall(pattern, row)
    if dates:
        return dates[-1] #date의 마지막 부분만 추출
    else:
        return None #else일 땐 None으로

# 'date' 열을 직접 변경하여 날짜를 추출한 값으로 대체 (일치하지 않는 형식의 데이터는 그대로 놔둠)
pro_new['date'] = pro_new['date'].apply(lambda x: pd.to_datetime(extract_date(x), \
                                                                 format='%Y-%m-%d').strftime('%Y-%m-%d') if extract_date(x) else x)

#수시 채용 -> 9999-12-31 형태로 변경
pro_new.loc[pro_new['date'].str.contains('수시 채용'), 'date'] = '9999-12-31'

# 9999-12-31 형태를 제외한 나머지 데이터들 값 확인(잘 바뀌었는지)
set(pro_new['date'])-set(pro_new.loc[pro_new['date'].str.contains('9999-12-31'), 'date'])

pro_new = pro_new.rename(columns = {'date' : 'date_until'})

career 컬럼 변경(ok)

In [127]:
pro_new.loc[pro_new['career'].str.contains('경력 무관'), 'career'] = '-1'
pro_new.loc[pro_new['career'].str.contains('신입'), 'career'] = '0'

selected_values = pro_new.loc[pro_new['career'].str.contains('\d ~ \d년'), 'career']
processed_values = [i[0].split('~')[0].replace(' ', '') for i in selected_values.to_list()]
pro_new.loc[selected_values.index, 'career'] = processed_values

#job_site_code 컬럼 추가, value = 3
pro_new['job_site_code'] = '3'

required, preferred_skill, welfare 컬럼 합치기

In [191]:
pro_new.columns

import numpy as np

#pd.set_option('display.max_rows', 15)
pro_new['combined'] = pro_new['required'] + ', ' + pro_new['preferred_skill'] + ', ' + pro_new['welfare']
pro_new['combined'] = pro_new['combined'].replace(np.nan, None) #NaN 값을 None으로 대체
pro_new = pro_new.drop(['welfare'], axis=1)
pro_new = pro_new.rename(columns = {'combined' : 'required'})

Index(['company_name', 'title', 'career', 'date', 'tech_stack', 'description',
       'company_description', 'required', 'preferred_skill', 'welfare',
       'job_site_code'],
      dtype='object')

company_description열에서 비어있는 값 None으로 채우기

In [241]:
import numpy as np

pro_new['company_description'] = pro_new['company_description'].replace(np.nan, None)

In [279]:
import pickle

with open("./pro_preprocess.pkl", "wb") as f:
    pickle.dump(pro_new, f)

tech_stack -> 리스트 풀어서 넣기

In [312]:
pro_new['tech_stack']

# 리스트를 문자열로 변환하는 함수 정의
def list_to_string(lst):
    if lst is None:
        return None
    else:
        return ', '.join(lst)

# 'tech_stack' 열의 모든 요소를 문자열로 변환
pro_new['tech_stack'] = pro_new['tech_stack'].apply(list_to_string)

0                                                    None
1                                                    None
2              [CI/CD, GCP(Google Cloud Platform), Azure]
3                                                    None
4                                         [Linux, C, C++]
                              ...                        
1191           [Java, Spring, PostgreSQL, GeoServer, ES6]
1192            [Java, MySQL, Redis, Spring, Git, Vue.js]
1193    [Java, MariaDB, Redis, AWS ElastiCache, AWS MS...
1194                                [MySQL, PHP, Laravel]
1195                [HTML, CSS, JavaScript, ReactJS, PHP]
Name: tech_stack, Length: 1196, dtype: object

In [326]:
pro_new

,job_site_code,company_name,title,career,date_until,tech_stack,description,company_description,required
0,3,웅진,[경력] 웹 시스템 설계,10,2024-05-31,None,"<blockquote class=""gt-blockquote""><strong clas...","SAP ERP ,AWS/MS클라우드, 세일즈포스, 그룹웨어 등 IT컨설팅 및 대외...",None
1,3,웅진,로우코드(Low-Code Platform) 웹 기반 시스템 개발,10,2024-05-30,None,"<blockquote class=""gt-blockquote""><strong clas...","SAP ERP ,AWS/MS클라우드, 세일즈포스, 그룹웨어 등 IT컨설팅 및 대외...",None
2,3,팀프레시,[팀프레시] 아키텍처팀 아키텍트 담당자,3,9999-12-31,"CI/CD, GCP(Google Cloud Platform), Azure","<span class=""colour"" style=""color:rgb(38, 55, ...",물류대행,"\- 1가지 언어\(Java, Kotlin 등\)를 이용하여 서비스를 개발한 경험이..."
3,3,팀프레시,[팀프레시] 인프라팀 인프라 관리자,3,9999-12-31,None,"**ABOUT '팀프레시'**\r\n\r\n<span class=""colour"" s...",물류대행,* 인프라 관리 경력 3년 이상 \~ 5년 이하인 자\r\n* 운전면허 2종 보통 ...
4,3,콘텔라,5G 기지국 소프트웨어 개발,-1,9999-12-31,"Linux, C, C++",* 근무부서 - AN개발그룹\r\n* 5G 기지국 MAC/스케줄러 소프트웨어 개발\...,응용/시스템 소스트웨어 개발 및 공급,* 대졸이상(4년제)\r\n* C Program 능숙자\r\n* 리눅스 기반 C/C...
...,...,...,...,...,...,...,...,...,...
1191,3,포스웨이브,GIS 웹 개발자,2,9999-12-31,"Java, Spring, PostgreSQL, GeoServer, ES6",* GIS 기반의 웹 시스템 개발 (주 SI)\r\n* 자사 공간빅데이터 솔루션과 ...,GeoWave(공간빅데이터솔루션),* Java Spring 프레임워크를 이용한 웹 개발 경험\r\n* 다양한 RDBM...
1192,3,비팩토리,JAVA 백엔드 개발자 (서버),3,9999-12-31,"Java, MySQL, Redis, Spring, Git, Vue.js","<span class=""colour"" style=""color:rgb(0, 0, 0)...",YG셀렉트 내 커스텀 서비스,"<span class=""colour"" style=""color:rgb(0, 0, 0)..."
1193,3,비즈톡,카카오/네이버 서비스 연동 서버/백엔드 개발자,5,9999-12-31,"Java, MariaDB, Redis, AWS ElastiCache, AWS MSK...",* 카카오톡 시스템 연동 비즈메시지(알림톡/친구톡) 발송 서버 개발.\r\n* 네이...,카카오 비즈메시지,* 서버 / 백엔드 개발자로서 실무 5년차 이상\r\n* Git 버전 관리 시스템 ...
1194,3,Xinics,백엔드 웹 개발자,-1,9999-12-31,"MySQL, PHP, Laravel","* 하버드, 예일 등 북미 아이비리그 대학 대다수에서 사용하는 Canvas를 기반으...","LearningX, CommonsEX, LectureDeck, ReadyStream...",* PHP 개발에 능숙하신 분\r\n* PHP Framework(Laravel) 기...


이모지 제거 및 컬럼 순서 변경

In [325]:
pro_new['required'] = pro_new['required'].apply(lambda x: x.encode('cp949', 'ignore').decode('cp949') if x else None)
pro_new['company_description'] = pro_new['company_description'].apply(lambda x: x.encode('cp949', 'ignore').decode('cp949') if x else None)
pro_new['description'] = pro_new['description'].apply(lambda x: x.encode('cp949', 'ignore').decode('cp949') if x else None)

pro_new = pro_new[['job_site_code', 'company_name', 'title', 'career', 'date_until', 'tech_stack',
       'description', 'company_description',  'required']]

마지막 pkl 파일

In [327]:
#import pickle

#with open("./final_pro.pkl", "rb") as f:
#    pickle.dump(pro_new, f)

Insert data into mysql

In [2]:
import pickle

with open("./data/final_pro.pkl", "rb") as f:
    final = pickle.load(f)

In [3]:
import pymysql
con =pymysql.connect(host='HostIP', user='UserID', password='UserPW', db='db', charset='utf8')
cur = con.cursor()

In [4]:
sql = """INSERT INTO job_data (job_site_code, company_name, title, career, date_until, tech_stack, description, company_description, required) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [5]:
from tqdm import tqdm

for idx, row in tqdm(final.iterrows()):
    #print(row)
    #print(type(row))
    #print(type(tuple(row)))
    try:
        cur.execute(sql, tuple(row.values))
    except Exception as e:
        print(e)

1196it [00:10, 117.30it/s]


In [6]:
con.commit()